In [1]:
import torch
import numpy as np
from torch.distributions import normal
from torch.distributions import multivariate_normal
import csv
import pandas as pd
import math
from random import randint

LABEL_INDEX = 25
ID_CONST = 24
PPG_INDEX = 19

def fix_names(df):
    index = 0
    for name in df['Player']:
        names = name.split("\\")
        df['Player'].values[index] = names[0]
        index += 1

def fix_nan(df):
    for s in df:
        if s != 'Player':
            index = 0
            for value in df[s]:
                if math.isnan(value):
                    #print("s = ", s)
                    #print("index = ", index)
                    #print("\t prev value = ", df1[s].values[index])
                    df[s].values[index] = 0.0
                    #print("\t new value = ", df1[s].values[index])
                index += 1
            
def get_dataTensors(df):
    _vec = []
    _lab = []

    for x, row in df.iterrows():
        x_vec = []
        label = int(row[LABEL_INDEX])
        _lab.append(label)
        for i in range(len(row)):
            data = row[i]
            if i != len(row)-1 and i != 0:
                x_vec.append(data)
        _vec.append(x_vec)
    
    return torch.tensor(_vec), torch.tensor(_lab)


# tiers for deciding popularity
tier1 = ["Lebron James", "Kevin Durant", "Giannis Antetokounmpo", "Stephen Curry", "Kawhi Leonard", "James Harden", 
          "Nikola Jokic", "Luka Doncic", "Damian Lillard", "Anthony Davis", "Joel Embiid"]

tier2 = ["Zion Williamson", "Trae Young", "Jimmy Butler", "Jayson Tatum", "Zach Lavine", "Devin Booker",
         "Donovon Mitchell","Kyrie Irving", "Paul George", "Ja Morant", "Klay Thompson", "Chris Paul"]

tier3 = ["Bradley Beal", "Khris Middleton", "Jaylen Brown", "Russell Westbrook", "Deandre Ayton",
        "Bam Adebayo", "Rudy Gobert", "Julius Randle", "Lamelo Ball", "Ben Simmons", "Brandon Ingram", 
        "Karl-Anthony Towns", "Jamal Murray", "DeMar DeRozan"]

tier4 = ["Domantas Sabonis", "Nikola Vucevic", "Kyle Lowry", "Pascal Siakam", "Jrue Holiday", "Victor Oladipo", 
         "De'Aaron Fox", "Buddy Hield", "Kemba Walker", "Kristaps Porziņģis", "CJ McCollum", "D'Angelo Russell", ]

tier5 = ["LaMarcus Aldridge", "Lonzo Ball", "Tobias Harris", "Draymond Green", "Shai Gilgeous-Alexander",
        "Dejounte Murray"]

df1 = pd.read_csv("nba2020stats.csv")
df_adv1 = pd.read_csv("nba2020adv.csv")
df1

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT,FTA,FT%,TRB,AST,STL,BLK,TOV,PF,PTS
0,Steven Adams\adamsst01,C,26,OKC,63,63,26.7,4.5,7.6,0.592,...,1.9,3.2,0.582,9.3,2.3,0.8,1.1,1.5,1.9,10.9
1,Bam Adebayo\adebaba01,PF,22,MIA,72,72,33.6,6.1,11.0,0.557,...,3.7,5.3,0.691,10.2,5.1,1.1,1.3,2.8,2.5,15.9
2,LaMarcus Aldridge\aldrila01,C,34,SAS,53,53,33.1,7.4,15.0,0.493,...,3.0,3.6,0.827,7.4,2.4,0.7,1.6,1.4,2.4,18.9
3,Kyle Alexander\alexaky01,C,23,MIA,2,0,6.5,0.5,1.0,0.500,...,0.0,0.0,NaN,1.5,0.0,0.0,0.0,0.5,0.5,1.0
4,Nickeil Alexander-Walker\alexani01,SG,21,NOP,47,1,12.6,2.1,5.7,0.368,...,0.5,0.8,0.676,1.8,1.9,0.4,0.2,1.1,1.2,5.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,Trae Young\youngtr01,PG,21,ATL,60,60,35.3,9.1,20.8,0.437,...,8.0,9.3,0.860,4.3,9.3,1.1,0.1,4.8,1.7,29.6
525,Cody Zeller\zelleco01,C,27,CHO,58,39,23.1,4.3,8.3,0.524,...,2.1,3.1,0.682,7.1,1.5,0.7,0.4,1.3,2.4,11.1
526,Tyler Zeller\zellety01,C,30,SAS,2,0,2.0,0.5,2.0,0.250,...,0.0,0.0,NaN,2.0,0.0,0.0,0.0,0.0,0.0,1.0
527,Ante Žižić\zizican01,C,23,CLE,22,0,10.0,1.9,3.3,0.569,...,0.6,0.9,0.737,3.0,0.3,0.3,0.2,0.5,1.2,4.4


In [2]:
df1['PER'] = df_adv1['PER']
df1['TS%'] = df_adv1['TS%']
# df1['MP'] = df_adv1['MP']
# df1['USG%'] = df_adv1['USG%']
# df1['WS'] = df_adv1['WS']
# df1['VORP'] = df_adv1['VORP']

fix_names(df1)
fix_names(df_adv1)

df1.at[133,'Player'] = "Luka Doncic"
df1.at[260,'Player'] = "Nikola Jokic"
df1.at[490,'Player'] = "Nikola Vucevic"
df1.at[410,'Player'] = "Kristaps Porzingis"
#df1

In [3]:
# 2019-2020 team records
team_records = {'ATL':0.299 , "BRK":0.486 , "BOS":0.667 , "CHO":0.354 , "CHI":0.338 , "CLE":0.292 , "DAL":0.573 , 
                "DEN":0.630 , "DET":0.303 , "GSW":0.231 , "HOU":0.611 , "IND":0.616 , "LAC":0.681 , "LAL":0.732 , 
                "MEM":0.466 , "MIA":0.603 , "MIL":0.767 , "MIN":0.297 , "NOP":0.417 , "NYK":0.318 , "OKC":0.611 , 
                "ORL":0.292 , "PHI":0.681 , "PHO":0.708 , "POR":0.583 , "SAC":0.431 , "SAS":0.458 , 
                "TOR":0.736 , "UTA":0.611 , "WAS":0.347, "TOT": 0.6}

player_team_records = []

for team in df1["Tm"]:
    player_team_records.append(team_records[team])

df1["Team Win%"] = player_team_records
#df1

In [4]:
all_stars_2020 = ["LeBron James", "Anthony Davis", "Kawhi Leonard", "Luka Doncic", "James Harden", "Damian Lillard", 
             "Ben Simmons", "Nikola Jokic", "Jayson Tatum", "Chris Paul", "Russell Westbrook", "Domantas Sabonis", 
            "Giannis Antetokounmpo", "Joel Embiid", "Pascal Siakam", "Kemba Walker", "Trae Young", "Khris Middleton", 
            "Bam Adebayo", "Rudy Gobert", "Jimmy Butler", "Kyle Lowry", "Brandon Ingram", "Donovan Mitchell", 
            "Devin Booker"]

is_allstar_2020 = []
popularity = []

for player in df1['Player']:
    if player in all_stars_2020:
        is_allstar_2020.append(1)
        if player in tier1:
            popularity.append(100)
        elif player in tier2:
            popularity.append(randint(80,99))
        elif player in tier3:
            popularity.append(randint(60,79))
        elif player in tier4:
            popularity.append(randint(40,59))
        elif player in tier5:
            popularity.append(randint(20,39))
        else:
            popularity.append(randint(5,19))
    else:
        is_allstar_2020.append(0)
        if player in tier1:
            popularity.append(100)
        elif player in tier2:
            popularity.append(randint(80,99))
        elif player in tier3:
            popularity.append(randint(60,79))
        elif player in tier4:
            popularity.append(randint(40,59))
        elif player in tier5:
            popularity.append(randint(20,39))
        else:
            popularity.append(randint(5,19))

print("# of all stars 2020 = ", len(all_stars_2020))

df1["popularity"] = popularity
df1["all star"] = is_allstar_2020

df1 = df1.drop(['Tm', 'Pos'], axis=1)

fix_nan(df1)

#df1

# of all stars 2020 =  25


In [5]:
df1 = df1.drop(['G', 'GS', 'Age', 'TOV'], axis=1)
df1

,Player,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,...,AST,STL,BLK,PF,PTS,PER,TS%,Team Win%,popularity,all star
0,Steven Adams,26.7,4.5,7.6,0.592,0.0,0.0,0.333,4.5,7.5,...,2.3,0.8,1.1,1.9,10.9,20.5,0.604,0.611,13,0
1,Bam Adebayo,33.6,6.1,11.0,0.557,0.0,0.2,0.143,6.1,10.8,...,5.1,1.1,1.3,2.5,15.9,20.3,0.598,0.603,74,1
2,LaMarcus Aldridge,33.1,7.4,15.0,0.493,1.2,3.0,0.389,6.2,12.0,...,2.4,0.7,1.6,2.4,18.9,19.7,0.571,0.458,29,0
3,Kyle Alexander,6.5,0.5,1.0,0.500,0.0,0.0,0.000,0.5,1.0,...,0.0,0.0,0.0,0.5,1.0,4.7,0.500,0.603,15,0
4,Nickeil Alexander-Walker,12.6,2.1,5.7,0.368,1.0,2.8,0.346,1.1,2.8,...,1.9,0.4,0.2,1.2,5.7,8.9,0.473,0.417,17,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,Trae Young,35.3,9.1,20.8,0.437,3.4,9.5,0.361,5.7,11.4,...,9.3,1.1,0.1,1.7,29.6,23.9,0.595,0.299,86,1
525,Cody Zeller,23.1,4.3,8.3,0.524,0.3,1.3,0.240,4.0,7.0,...,1.5,0.7,0.4,2.4,11.1,18.8,0.576,0.354,12,0
526,Tyler Zeller,2.0,0.5,2.0,0.250,0.0,0.0,0.000,0.5,2.0,...,0.0,0.0,0.0,0.0,1.0,22.4,0.250,0.458,8,0
527,Ante Žižić,10.0,1.9,3.3,0.569,0.0,0.0,0.000,1.9,3.3,...,0.3,0.3,0.2,1.2,4.4,16.4,0.597,0.292,15,0


In [6]:
validation_vec, validation_lab = get_dataTensors(df1)

print("validation_vec[0] = ", validation_vec[0])
print("size^ = ", len(validation_vec[0]))
print("size validation_vec = ", len(validation_vec))
print("validation_lab[0] = ", validation_lab[0])
print("PPG for index 0 = ", validation_vec[0][PPG_INDEX].item())

validation_vec[0] =  tensor([26.7000,  4.5000,  7.6000,  0.5920,  0.0000,  0.0000,  0.3330,  4.5000,
         7.5000,  0.5940,  0.5930,  1.9000,  3.2000,  0.5820,  9.3000,  2.3000,
         0.8000,  1.1000,  1.9000, 10.9000, 20.5000,  0.6040,  0.6110, 13.0000])
size^ =  24
size validation_vec =  529
validation_lab[0] =  tensor(0)
PPG for index 0 =  10.899999618530273


In [7]:
df2 = pd.read_csv("nba2021stats.csv")
df_adv2 = pd.read_csv("nba2021adv.csv")
df2

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT,FTA,FT%,TRB,AST,STL,BLK,TOV,PF,PTS
0,Precious Achiuwa\achiupr01,PF,21,MIA,61,4,12.1,2.0,3.7,0.544,...,0.9,1.8,0.509,3.4,0.5,0.3,0.5,0.7,1.5,5.0
1,Jaylen Adams\adamsja01,PG,24,MIL,7,0,2.6,0.1,1.1,0.125,...,0.0,0.0,NaN,0.4,0.3,0.0,0.0,0.0,0.1,0.3
2,Steven Adams\adamsst01,C,27,NOP,58,58,27.7,3.3,5.3,0.614,...,1.0,2.3,0.444,8.9,1.9,0.9,0.7,1.3,1.9,7.6
3,Bam Adebayo\adebaba01,C,23,MIA,64,64,33.5,7.1,12.5,0.570,...,4.4,5.5,0.799,9.0,5.4,1.2,1.0,2.6,2.3,18.7
4,LaMarcus Aldridge\aldrila01,C,35,TOT,26,23,25.9,5.4,11.4,0.473,...,1.6,1.8,0.872,4.5,1.9,0.4,1.1,1.0,1.8,13.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,Delon Wright\wrighde01,SG-PG,28,TOT,63,39,27.7,3.8,8.2,0.463,...,1.6,2.0,0.802,4.3,4.4,1.6,0.5,1.3,1.2,10.2
536,Thaddeus Young\youngth01,PF,32,CHI,68,23,24.3,5.4,9.7,0.559,...,1.0,1.7,0.628,6.2,4.3,1.1,0.6,2.0,2.2,12.1
537,Trae Young\youngtr01,PG,22,ATL,63,63,33.7,7.7,17.7,0.438,...,7.7,8.7,0.886,3.9,9.4,0.8,0.2,4.1,1.8,25.3
538,Cody Zeller\zelleco01,C,28,CHO,48,21,20.9,3.8,6.8,0.559,...,1.8,2.5,0.714,6.8,1.8,0.6,0.4,1.1,2.5,9.4


In [8]:
df2['PER'] = df_adv2['PER']
df2['TS%'] = df_adv2['TS%']
# df2['MP'] = df_adv2['MP']
# df2['USG%'] = df_adv2['USG%']
# df2['WS'] = df_adv2['WS']
# df2['VORP'] = df_adv2['VORP']

fix_names(df2)
fix_names(df_adv2)

df2.at[129,'Player'] = "Luka Doncic"
df2.at[261,'Player'] = "Nikola Jokic"
df2.at[503,'Player'] = "Nikola Vucevic"
df2.at[412,'Player'] = "Kristaps Prozingis"

#df2

In [9]:
# 2020-21 team records
team_records = {'ATL':0.569 , "BRK":0.667 , "BOS":0.5 , "CHO":0.458 , "CHI":0.431 , "CLE":0.306 , "DAL":0.583 , 
                "DEN":0.653 , "DET":0.278 , "GSW":0.542 , "HOU":0.236 , "IND":0.427 , "LAC":0.653 , "LAL":0.583 , 
                "MEM":0.528 , "MIA":0.556 , "MIL":0.639 , "MIN":0.319 , "NOP":0.431 , "NYK":0.569 , "OKC":0.306 , 
                "ORL":0.292 , "PHI":0.681 , "PHO":0.708 , "POR":0.583 , "SAC":0.431 , "SAS":0.458 , 
                "TOR":0.375 , "UTA":0.722 , "WAS":0.427, "TOT":0.6}

player_team_records = []

for team in df2["Tm"]:
    player_team_records.append(team_records[team])

df2["Team Win%"] = player_team_records
#df2

In [10]:
all_stars_2021 = ["LeBron James", "Kevin Durant", "Anthony Davis", "Kawhi Leonard", "Luka Doncic", "James Harden", "Nikola Jokic",
            "Damian Lillard", "Chris Paul", "Domantas Sabonis", "Ben Simmons", "Jayson Tatum",
            "Stephen Curry", "Jaylen Brown", "Paul George", "Rudy Gobert", "Joel Embiid", "Devin Booker", 
             "Nikola Vucevic", "Julius Randle", "Donovan Mitchell", "Zach LaVine", "Mike Conley", "Zion Williamson",
            "Bradley Beal", "Kyrie Irving", "Giannis Antetokounmpo"]

is_allstar_2021 = []
popularity = []

for player in df2['Player']:
    if player in all_stars_2021:
        is_allstar_2021.append(1)
        if player in tier1:
            popularity.append(100)
        elif player in tier2:
            popularity.append(randint(80,99))
        elif player in tier3:
            popularity.append(randint(60,79))
        elif player in tier4:
            popularity.append(randint(40,59))
        elif player in tier5:
            popularity.append(randint(20,39))
        else:
            popularity.append(randint(5,19))
    else:
        is_allstar_2021.append(0)
        if player in tier1:
            popularity.append(100)
        elif player in tier2:
            popularity.append(randint(80,99))
        elif player in tier3:
            popularity.append(randint(60,79))
        elif player in tier4:
            popularity.append(randint(40,59))
        elif player in tier5:
            popularity.append(randint(20,39))
        else:
            popularity.append(randint(5,19))
            

df2["popularity"] = popularity        
df2["all star"] = is_allstar_2021

print("# of all stars 2021 = ", len(all_stars_2021))

df2 = df2.drop(['Tm', 'Pos'], axis=1)

fix_nan(df2)

#df2

# of all stars 2021 =  27


In [11]:
df2 = df2.drop(['G', 'GS', 'Age', 'TOV'], axis=1)
df2

,Player,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,...,AST,STL,BLK,PF,PTS,PER,TS%,Team Win%,popularity,all star
0,Precious Achiuwa,12.1,2.0,3.7,0.544,0.0,0.0,0.000,2.0,3.7,...,0.5,0.3,0.5,1.5,5.0,14.2,0.550,0.556,17,0
1,Jaylen Adams,2.6,0.1,1.1,0.125,0.0,0.3,0.000,0.1,0.9,...,0.3,0.0,0.0,0.1,0.3,-6.5,0.125,0.639,7,0
2,Steven Adams,27.7,3.3,5.3,0.614,0.0,0.1,0.000,3.3,5.3,...,1.9,0.9,0.7,1.9,7.6,15.1,0.596,0.431,18,0
3,Bam Adebayo,33.5,7.1,12.5,0.570,0.0,0.1,0.250,7.1,12.4,...,5.4,1.2,1.0,2.3,18.7,22.7,0.626,0.556,66,0
4,LaMarcus Aldridge,25.9,5.4,11.4,0.473,1.2,3.1,0.388,4.2,8.3,...,1.9,0.4,1.1,1.8,13.5,15.7,0.556,0.600,29,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,Delon Wright,27.7,3.8,8.2,0.463,1.0,2.7,0.372,2.8,5.5,...,4.4,1.6,0.5,1.2,10.2,16.3,0.562,0.600,14,0
536,Thaddeus Young,24.3,5.4,9.7,0.559,0.2,0.7,0.267,5.3,9.1,...,4.3,1.1,0.6,2.2,12.1,20.3,0.578,0.431,15,0
537,Trae Young,33.7,7.7,17.7,0.438,2.2,6.3,0.343,5.6,11.3,...,9.4,0.8,0.2,1.8,25.3,23.0,0.589,0.569,94,0
538,Cody Zeller,20.9,3.8,6.8,0.559,0.1,0.6,0.143,3.7,6.2,...,1.8,0.6,0.4,2.5,9.4,18.2,0.599,0.458,16,0


In [12]:
test_vec, test_lab = get_dataTensors(df2)

print("test_vec[0] = ", test_vec[0])
print("size^ = ", len(test_vec[0]))
print("size test_vec = ", len(test_vec))
print("test_lab[0] = ", test_lab[0])
print("PPG for index 0 = ", test_vec[0][PPG_INDEX].item())

test_vec[0] =  tensor([12.1000,  2.0000,  3.7000,  0.5440,  0.0000,  0.0000,  0.0000,  2.0000,
         3.7000,  0.5460,  0.5440,  0.9000,  1.8000,  0.5090,  3.4000,  0.5000,
         0.3000,  0.5000,  1.5000,  5.0000, 14.2000,  0.5500,  0.5560, 17.0000])
size^ =  24
size test_vec =  540
test_lab[0] =  tensor(0)
PPG for index 0 =  5.0


In [13]:
df3 = pd.read_csv("nba2019stats.csv")
df_adv3 = pd.read_csv("nba2019adv.csv")
df3

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT,FTA,FT%,TRB,AST,STL,BLK,TOV,PF,PTS
0,Álex Abrines\abrinal01,SG,25,OKC,31,2,19.0,1.8,5.1,0.357,...,0.4,0.4,0.923,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,Quincy Acy\acyqu01,PF,28,PHO,10,0,12.3,0.4,1.8,0.222,...,0.7,1.0,0.700,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,Jaylen Adams\adamsja01,PG,22,ATL,34,1,12.6,1.1,3.2,0.345,...,0.2,0.3,0.778,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,Steven Adams\adamsst01,C,25,OKC,80,80,33.4,6.0,10.1,0.595,...,1.8,3.7,0.500,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,Bam Adebayo\adebaba01,C,21,MIA,82,28,23.3,3.4,5.9,0.576,...,2.0,2.8,0.735,7.3,2.2,0.9,0.8,1.5,2.5,8.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,Trae Young\youngtr01,PG,20,ATL,81,81,30.9,6.5,15.5,0.418,...,4.2,5.1,0.829,3.7,8.1,0.9,0.2,3.8,1.7,19.1
526,Cody Zeller\zelleco01,C,26,CHO,49,47,25.4,3.9,7.0,0.551,...,2.3,2.9,0.787,6.8,2.1,0.8,0.8,1.3,3.3,10.1
527,Tyler Zeller\zellety01,C,29,TOT,6,1,15.5,2.7,5.0,0.533,...,2.3,3.0,0.778,4.0,0.7,0.2,0.5,0.7,3.3,7.7
528,Ante Žižić\zizican01,C,22,CLE,59,25,18.3,3.1,5.6,0.553,...,1.6,2.2,0.705,5.4,0.9,0.2,0.4,1.0,1.9,7.8


In [14]:
df3['PER'] = df_adv3['PER']
df3['TS%'] = df_adv3['TS%']

fix_names(df3)
fix_names(df_adv3)

df3.at[141, 'Player'] = "Luka Doncic"
df3.at[268, 'Player'] = "Nikola Jokic"
df3.at[352, 'Player'] = "Nikola Mirotic"
df3.at[492, 'Player']= "Nikola Vucevic"
df3.at[380, 'Player'] = "Jusuf Nurkic"

In [15]:
# 2018-2019 Team Records
team_records = {'ATL':0.354 , "BRK":0.512 , "BOS":0.598 , "CHO":0.476 , "CHI":0.268 , "CLE":0.232 , "DAL":0.402 , 
                "DEN":0.659 , "DET":0.5 , "GSW":0.695 , "HOU":0.646 , "IND":0.585 , "LAC":0.585 , "LAL":0.451 , 
                "MEM":0.402 , "MIA":0.476 , "MIL":0.732 , "MIN":0.439 , "NOP":0.402 , "NYK":0.207 , "OKC":0.598 , 
                "ORL":0.512 , "PHI":0.622 , "PHO":0.232 , "POR":0.646 , "SAC":0.476 , "SAS":0.585 , 
                "TOR":0.707 , "UTA":0.61 , "WAS":0.39, "TOT":0.6}

player_team_records = []

for team in df3["Tm"]:
    player_team_records.append(team_records[team])

df3["Team Win%"] = player_team_records
#df3

In [16]:
all_stars_2019 = ["LeBron James", "Kevin Durant", "Anthony Davis", "Kawhi Leonard", "James Harden", 
                  "Nikola Jokic", "Damian Lillard", "Ben Simmons", "Stephen Curry", 
                  "Paul George", "Joel Embiid", "Nikola Vucevic", "Bradley Beal", "Kyrie Irving", 
                  "Giannis Antetokounmpo", "Klay Thompson", "Kemba Walker", "Khris Middleton", "Blake Griffin",
                 "Russell Westbrook", "D'Angelo Russell", "LaMarcus Aldridge", "Karl-Anthony Towns", "Kyle Lowry"]

is_allstar_2019 = []
popularity = []

for player in df3['Player']:
    if player in all_stars_2019:
        is_allstar_2019.append(1)
        if player in tier1:
            popularity.append(100)
        elif player in tier2:
            popularity.append(randint(80,99))
        elif player in tier3:
            popularity.append(randint(60,79))
        elif player in tier4:
            popularity.append(randint(40,59))
        elif player in tier5:
            popularity.append(randint(20,39))
        else:
            popularity.append(randint(5,19))
    else:
        is_allstar_2019.append(0)
        if player in tier1:
            popularity.append(100)
        elif player in tier2:
            popularity.append(randint(80,99))
        elif player in tier3:
            popularity.append(randint(60,79))
        elif player in tier4:
            popularity.append(randint(40,59))
        elif player in tier5:
            popularity.append(randint(20,39))
        else:
            popularity.append(randint(5,19))
    
df3["popularity"] = popularity
df3["all star"] = is_allstar_2019

print("# of all stars 2019 = ", len(all_stars_2019))

df3 = df3.drop(['Tm', 'Pos'], axis=1)

fix_nan(df3)

# of all stars 2019 =  24


In [17]:
df3 = df3.drop(['G', 'GS', 'Age', 'TOV'], axis=1)
df3

,Player,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,...,AST,STL,BLK,PF,PTS,PER,TS%,Team Win%,popularity,all star
0,Álex Abrines,19.0,1.8,5.1,0.357,1.3,4.1,0.323,0.5,1.0,...,0.6,0.5,0.2,1.7,5.3,6.3,0.507,0.598,17,0
1,Quincy Acy,12.3,0.4,1.8,0.222,0.2,1.5,0.133,0.2,0.3,...,0.8,0.1,0.4,2.4,1.7,2.9,0.379,0.232,15,0
2,Jaylen Adams,12.6,1.1,3.2,0.345,0.7,2.2,0.338,0.4,1.1,...,1.9,0.4,0.1,1.3,3.2,7.6,0.474,0.354,16,0
3,Steven Adams,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,...,1.6,1.5,1.0,2.6,13.9,18.5,0.591,0.598,17,0
4,Bam Adebayo,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,...,2.2,0.9,0.8,2.5,8.9,17.9,0.623,0.476,60,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,Trae Young,30.9,6.5,15.5,0.418,1.9,6.0,0.324,4.6,9.6,...,8.1,0.9,0.2,1.7,19.1,14.6,0.543,0.354,86,0
526,Cody Zeller,25.4,3.9,7.0,0.551,0.1,0.4,0.273,3.8,6.6,...,2.1,0.8,0.8,3.3,10.1,17.0,0.587,0.476,16,0
527,Tyler Zeller,15.5,2.7,5.0,0.533,0.0,0.2,0.000,2.7,4.8,...,0.7,0.2,0.5,3.3,7.7,15.3,0.512,0.600,8,0
528,Ante Žižić,18.3,3.1,5.6,0.553,0.0,0.0,0.000,3.1,5.6,...,0.9,0.2,0.4,1.9,7.8,8.5,0.484,0.232,18,0


In [18]:
train_vec, train_lab = get_dataTensors(df3)

print("train_vec[0] = ", train_vec[0])
print("size^ = ", len(train_vec[0]))
print("size train_vec = ", len(train_vec))
print("train_lab[0] = ", train_lab[0])
print("PPG for index 0 = ", train_vec[0][PPG_INDEX].item())

train_vec[0] =  tensor([19.0000,  1.8000,  5.1000,  0.3570,  1.3000,  4.1000,  0.3230,  0.5000,
         1.0000,  0.5000,  0.4870,  0.4000,  0.4000,  0.9230,  1.5000,  0.6000,
         0.5000,  0.2000,  1.7000,  5.3000,  6.3000,  0.5070,  0.5980, 17.0000])
size^ =  24
size train_vec =  530
train_lab[0] =  tensor(0)
PPG for index 0 =  5.300000190734863


In [19]:
def fe0(X):
    return_x = []
    for x in X:
        return_x.append(x[PPG_INDEX])
    return torch.tensor(return_x)

def fe1(X):
    return X;

fe = [fe0,fe1]

In [20]:
class GNB:
    '''
    Constructor
    input: uni - set as True for univariate model
           ID_FE - set as 2 for fe2, otherwise don't change
    '''
    def __init__ (self, uni=True, ID_FE=0):
        self.uni = uni
        # do not change these
        self.prior = None       # list of the priors. Made from empirical counts
        self.dists = None       # list of Gaussian distributions likelihood. size = number of classes
        self.num_c = 0          # number of classes
        self.ID_FE = max(0 if self.uni else 1, ID_FE) # select feature extractor
    
    '''
    Train your model
    input: X - training data
           y - training labels
           uni - set as True for univariate model
    '''
    def train(self, X, y):
        self.dists = list()
        uniq_y = torch.unique(y)                      # get all labels
        self.prior = torch.zeros(uniq_y.shape[0])     # initialize prior P(Y) to zeros
        self.num_c = uniq_y.shape[0]                  # number of classes  
        
        print("self.dists = ", self.dists)
        print("uniq_y = ", uniq_y)
        print("self.prior = ", self.prior)
        print("self.num_c = ", self.num_c)
        print(" ")
        
        for idx in range(self.num_c):           # for each label
            print("counter = ", idx)
            mask = torch.eq(y,idx)
            x_by_label = X[mask]           # select all observations with label
            num_of_x = x_by_label.size()[0]
            
            self.prior[idx] = num_of_x/y.size()[0]            ##### TODO : Estimate your model's prior P(Y) (parts a, b, and c)
            # raise NotImplementedError           ##### TODO : Remove this line before running
            print("self.prior[" + str(idx) + "] = " + str(self.prior[idx]))
            print("mask = ", mask)
            print("x_by_label = ", x_by_label)
            #print("num_of_x = ", num_of_x)
            print("x_by_label.size() = ", x_by_label.size())
            x = fe[self.ID_FE](x_by_label)      # extract the feature vector
            print("x feature = ", x)
            print("x feature size = ", x.size())
            if self.uni: ##### Univariate gaussian (parts a and b)

                ########### TODO: Estimate the univariate Gaussian conditional distributions (parts a and b)           
                ##### Hint: Compute the empirical mean and variance. Then you can call PyTorch's "Normal" 
                #####       function with the correct arguments to build the distribution. We have already 
                #####       imported the relevant function for you. You can refer to PyTorch's official       
                #####       documentation for more info. 
                x_mean = x.mean()                  ##### TODO : calculate the mean value 
                print("x feature mean = ", x_mean)
                x_var = x.var()                  ##### TODO : calculate the variance
                print("x feature variance = ", x_var)
                gaussian_dist = normal.Normal(x_mean, torch.sqrt(x_var))          ##### TODO : replace 'None' with distribution's constructor
                print("gaussian dist = ", gaussian_dist)
                # raise NotImplementedError     ##### TODO : remove this line before running

            else: ##### Multivariate Gaussian. (part c)

                ########### TODO: Estimate the multivariate Gaussian conditional distributions (part c)           
                ##### Hint: In this part, you will call PyTorch's "MultivariateNormal" function to build    
                #####       a multivariate normal distribution. The function is already imported.     
                xT = x.T
                print("xT = ", xT)
                print("xT size = ", xT.size())
                n_sample = x.shape[0]
                print("num samples = ", n_sample)
                constant = 0.02
                x_mean = torch.mean(xT, dim=1)             ##### TODO : estimate the mean vector
                print("xT mean = ", x_mean)
                print("xT mean size = ", x_mean.size())
                x_minus_mu = x - x_mean
                print("x - mu = ", x_minus_mu)
                print("x - mu size = ", x_minus_mu.size())
                x_cov = torch.matmul(x_minus_mu.T,  x_minus_mu) / n_sample                  ##### TODO : estimate the covariance matrix
                id_matrix = torch.tensor([constant]*ID_CONST)
                small_constant_id_matrix = torch.diag(id_matrix)
                x_cov = x_cov + small_constant_id_matrix
                print("x covariance matrix = ", x_cov)
                print("x cov size = ", x_cov.size())
                gaussian_dist = multivariate_normal.MultivariateNormal(x_mean, x_cov)       ##### TODO : replace 'None' with distribution's constructor
                # raise NotImplementedError     ##### TODO : remove this line before running
                print("gaussian_dist = ", gaussian_dist)
            print("----------------------------------------------------")
            
            self.dists.append(gaussian_dist)
        return

    '''
        Use Bayes rule to predict on one sample
    '''
    def predict(self, x):
        #print("\t inside predict function")
        #print("\t x = ", x)
        x = fe[self.ID_FE](torch.reshape(x,(1,-1)))
        #print("\t x reshape = ", x)
        result = []
        #print("\t self.num_c = ", self.num_c)
        for i in range(self.num_c):
            term1 = self.dists[i].log_prob(x)
            term2 = torch.log(self.prior[i])
            #print("\t \t term1, term2 = " + str(term1) + ", " + str(term2))
            result.append(term1+term2)
        #result = [self.dists[i].log_prob(x) + torch.log(self.prior[i]) for i in range(self.num_c)]
        #print("\t result = ", result)
        result_tensor = torch.tensor(result)
        #print("\t result tensor = ", result_tensor)
        to_return = torch.argmax(result_tensor)
        #print("\t return = ", to_return)
        return to_return

    '''
        Evaluate classification accuracy
    '''
    def evaluate(self, X, y, df):
        correct = 0
        wrong = 0
        num_allstars = 0
        num_allstars_correct = 0
        correct_allstars = []
        wrong_allstars = []
        
        num_nonstars = 0
        num_nonstars_correct = 0
        correct_nonstars = []
        wrong_nonstars = []
        
        for i in range(X.shape[0]):
            prediction = self.predict(X[i])
            actual = y[i]
            if actual == 1:
                num_allstars += 1
                if prediction == 1:
                    num_allstars_correct += 1
                    correct_allstars.append(df['Player'].values[i])
                else:
                    wrong_allstars.append(df['Player'].values[i])
            if actual == 0:
                num_nonstars += 1
                if prediction == 0:
                    num_nonstars_correct += 1
                    correct_nonstars.append(df['Player'].values[i])
                else:
                    wrong_nonstars.append(df['Player'].values[i])
#             if prediction == actual:
#                 correct = correct + 1
#             else:
#                 #print("\t \t ^incorrect prediction at index = ", i)
#                 print("Player = ", df['Player'].values[i])
#                 print("\t Prediction all star? ", prediction.item())
#                 print("\t actual all star?", actual.item())
#                 print(" ")
#                 wrong += 1
#         print('The classification accuracy is {:.3f}'.format(correct/X.shape[0]))
#         print("wrong = ", wrong)

        print("Num of actual all stars: ", num_allstars)
        print("Num of correctly predicted all stars: ", num_allstars_correct)
        print("Accuracy of predicting actual all stars to be all stars: ", num_allstars_correct/num_allstars)
        print(" ")
        print("These are all the actual all stars that were predicted to be all stars: ")
        print(correct_allstars)
        print(" ")
        print("These are all the actual all stars that were predicted to NOT be all stars: ")
        print(wrong_allstars)
        print(" ")
        print("Num of actual non all stars: ", num_nonstars)
        print("Num of correctly predicted non all stars: ", num_nonstars_correct)
        print("Accuracy of predicting actual non all stars to not be all stars: ", num_nonstars_correct/num_nonstars)
        print(" ")
        print("These are all the actual non all stars that were predicted to be all stars: ")
        print(wrong_nonstars)
        return

In [21]:
# df1
# #df2
# index = 0
# for v in df1['all star']:
#     if v == 1:
#         print(index)
#     index+=1
# print("2019-2020 all star indicies")
# index = 0
# i = []
# for a in df1['all star']:
#     if a == 1:
#         i.append(index)
#     index += 1
# print(i)

In [22]:
model_multi = GNB(uni=False)
model_multi.train(train_vec, train_lab)

self.dists =  []
uniq_y =  tensor([0, 1])
self.prior =  tensor([0., 0.])
self.num_c =  2
 
counter =  0
self.prior[0] = tensor(0.9547)
mask =  tensor([ True,  True,  True,  True,  True,  True,  True, False,  True,  True,
         True,  True,  True,  True,  True,  True,  True, False,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True, False,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,

In [23]:
# 2018-19 season
print("Multivariate Training Data: ")    
print(" ")
model_multi.evaluate(train_vec, train_lab, df3)
print(" ")

Multivariate Training Data: 
 
Num of actual all stars:  24
Num of correctly predicted all stars:  24
Accuracy of predicting actual all stars to be all stars:  1.0
 
These are all the actual all stars that were predicted to be all stars: 
['LaMarcus Aldridge', 'Giannis Antetokounmpo', 'Bradley Beal', 'Stephen Curry', 'Anthony Davis', 'Kevin Durant', 'Joel Embiid', 'Paul George', 'Blake Griffin', 'James Harden', 'Kyrie Irving', 'LeBron James', 'Nikola Jokic', 'Kawhi Leonard', 'Damian Lillard', 'Kyle Lowry', 'Khris Middleton', "D'Angelo Russell", 'Ben Simmons', 'Klay Thompson', 'Karl-Anthony Towns', 'Nikola Vucevic', 'Kemba Walker', 'Russell Westbrook']
 
These are all the actual all stars that were predicted to NOT be all stars: 
[]
 
Num of actual non all stars:  506
Num of correctly predicted non all stars:  491
Accuracy of predicting actual non all stars to not be all stars:  0.9703557312252964
 
These are all the actual non all stars that were predicted to be all stars: 
['Deandre A

In [24]:
# for i,row in df1.iterrows():
#     print("index = ", i)
#     print(row[0])
#     print(" ")

In [25]:
# 2019-20 season
print("Multivariate Validation Data: ") 
print(" ")
model_multi.evaluate(validation_vec, validation_lab, df1)
print(" ")

Multivariate Validation Data: 
 
Num of actual all stars:  25
Num of correctly predicted all stars:  22
Accuracy of predicting actual all stars to be all stars:  0.88
 
These are all the actual all stars that were predicted to be all stars: 
['Bam Adebayo', 'Giannis Antetokounmpo', 'Devin Booker', 'Jimmy Butler', 'Anthony Davis', 'Luka Doncic', 'Joel Embiid', 'Rudy Gobert', 'James Harden', 'Brandon Ingram', 'LeBron James', 'Nikola Jokic', 'Kawhi Leonard', 'Damian Lillard', 'Kyle Lowry', 'Khris Middleton', 'Chris Paul', 'Domantas Sabonis', 'Ben Simmons', 'Jayson Tatum', 'Russell Westbrook', 'Trae Young']
 
These are all the actual all stars that were predicted to NOT be all stars: 
['Donovan Mitchell', 'Pascal Siakam', 'Kemba Walker']
 
Num of actual non all stars:  504
Num of correctly predicted non all stars:  485
Accuracy of predicting actual non all stars to not be all stars:  0.9623015873015873
 
These are all the actual non all stars that were predicted to be all stars: 
['LaMarcu

In [26]:
# 2020-21 season
print("Multivariate Test Data: ") 
print(" ")
model_multi.evaluate(test_vec, test_lab, df2)
print(" ")

Multivariate Test Data: 
 
Num of actual all stars:  27
Num of correctly predicted all stars:  26
Accuracy of predicting actual all stars to be all stars:  0.9629629629629629
 
These are all the actual all stars that were predicted to be all stars: 
['Giannis Antetokounmpo', 'Bradley Beal', 'Devin Booker', 'Jaylen Brown', 'Stephen Curry', 'Anthony Davis', 'Luka Doncic', 'Kevin Durant', 'Joel Embiid', 'Paul George', 'Rudy Gobert', 'James Harden', 'Kyrie Irving', 'LeBron James', 'Nikola Jokic', 'Zach LaVine', 'Kawhi Leonard', 'Damian Lillard', 'Donovan Mitchell', 'Chris Paul', 'Julius Randle', 'Domantas Sabonis', 'Ben Simmons', 'Jayson Tatum', 'Nikola Vucevic', 'Zion Williamson']
 
These are all the actual all stars that were predicted to NOT be all stars: 
['Mike Conley']
 
Num of actual non all stars:  513
Num of correctly predicted non all stars:  491
Accuracy of predicting actual non all stars to not be all stars:  0.9571150097465887
 
These are all the actual non all stars that were